# Creating a Spell Checker

In [144]:
import pandas as pd
import numpy as np
import tensorflow as tf
import os
from os import listdir
from os.path import isfile, join
from collections import namedtuple
from tensorflow.python.layers.core import Dense
from tensorflow.python.ops.rnn_cell_impl import _zero_state_tensors
import time
import re
from sklearn.model_selection import train_test_split

## Loading the Data

In [66]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [77]:
df = pd.read_csv('./data/dataset.csv')
df = df.dropna()

In [81]:
df[:10]

,noise_sentence,label
0,The Moon is Erath only natural xatellite,The Moon is Earth only natural satellite
1,At about the diameter of EartE comparable to t...,At about the diameter of Earth comparable to t...
2,The Moon is a object that formed a differentia...,The Moon is a object that formed a differentia...
3,It lacks ayn signifciant atmosphere hydrospher...,It lacks any significant atmosphere hydrospher...
4,Its surface rgavity is abyut of Eqrth g,Its surface gravity is about of Earth g
5,Jupitr moom Io is the onlj satellit2 in the Co...,Jupiter moon Io is the only satellite in the S...
6,Orbitigg Earth at an average distance of km Di...,Orbiting Earth at an average distance of km mi...
7,The Moon orbit around Earth has a sidereal per...,The Moon orbit around Earth has a sidereal per...
8,During rach synodic priod of days the amount o...,During each synodic period of days the amount ...
9,The Hoon is tidally locked to Eartu which mean...,The Moon is tidally locked to Earth which mean...


In [82]:
samples = df.values.tolist()
print(f"There are {len(samples)} samples.")

There are 225857 samples.


In [83]:
samples[:10]

[['The Moon is Erath only natural xatellite',
  'The Moon is Earth only natural satellite'],
 ['At about the diameter of EartE comparable to teh width of Australia it 7s thd fifth largest satellite in the Splar Systrm the .argest satellite in the Solar System relQtive to its major planet and lrger than ay known dwarf planet',
  'At about the diameter of Earth comparable to the width of Australia it is the fifth largest satellite in the Solar System the largest satellite in the Solar System relative to its major planet and larger than any known dwarf planet'],
 ['The Moon is a object that formed a differentiated rocky body making it a satellite planet under 4he geophysical definitions of the trm',
  'The Moon is a object that formed a differentiated rocky body making it a satellite planet under the geophysical definitions of the term'],
 ['It lacks ayn signifciant atmosphere hydrosphere or magnetic fiKld',
  'It lacks any significant atmosphere hydrosphere or magnetic field'],
 ['Its su

In [85]:
# Create a dictionary to convert the vocabulary (characters) to integers
vocab_to_int = {}
count = 0
for sample in samples:
    noisy_sen, sen = sample
    
    for character in sen:
        if character not in vocab_to_int:
            vocab_to_int[character] = count
            count += 1

    for character in noisy_sen:
        if character not in vocab_to_int:
            vocab_to_int[character] = count
            count += 1

# Add special tokens to vocab_to_int
codes = ['<PAD>','<EOS>','<GO>']
for code in codes:
    vocab_to_int[code] = count
    count += 1

In [86]:
# Check the size of vocabulary and all of the values
vocab_size = len(vocab_to_int)
print("The vocabulary contains {} characters.".format(vocab_size))
print(sorted(vocab_to_int))

The vocabulary contains 78 characters.
[' ', ',', '-', '.', '/', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', ';', '<', '<EOS>', '<GO>', '<PAD>', '>', '?', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', '[', '`', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '{']


In [87]:
# Convert sentences to integers
int_sentences = []
int_noise_sentences = []

for sample in samples:
    noisy_sen, sentence = sample
  
    int_sentence = []
    for character in sentence:
        int_sentence.append(vocab_to_int[character])
    int_sentences.append(int_sentence)

    int_noise_sentence = []
    for character in sentence:
        int_noise_sentence.append(vocab_to_int[character])
    int_noise_sentences.append(int_noise_sentence)

In [88]:
print(int_sentences[:3])
print(int_noise_sentences[:3])

[[0, 1, 2, 3, 4, 5, 5, 6, 3, 7, 8, 3, 9, 10, 11, 12, 1, 3, 5, 6, 13, 14, 3, 6, 10, 12, 15, 11, 10, 13, 3, 8, 10, 12, 2, 13, 13, 7, 12, 2], [17, 12, 3, 10, 18, 5, 15, 12, 3, 12, 1, 2, 3, 19, 7, 10, 20, 2, 12, 2, 11, 3, 5, 21, 3, 9, 10, 11, 12, 1, 3, 22, 5, 20, 23, 10, 11, 10, 18, 13, 2, 3, 12, 5, 3, 12, 1, 2, 3, 24, 7, 19, 12, 1, 3, 5, 21, 3, 17, 15, 8, 12, 11, 10, 13, 7, 10, 3, 7, 12, 3, 7, 8, 3, 12, 1, 2, 3, 21, 7, 21, 12, 1, 3, 13, 10, 11, 25, 2, 8, 12, 3, 8, 10, 12, 2, 13, 13, 7, 12, 2, 3, 7, 6, 3, 12, 1, 2, 3, 26, 5, 13, 10, 11, 3, 26, 14, 8, 12, 2, 20, 3, 12, 1, 2, 3, 13, 10, 11, 25, 2, 8, 12, 3, 8, 10, 12, 2, 13, 13, 7, 12, 2, 3, 7, 6, 3, 12, 1, 2, 3, 26, 5, 13, 10, 11, 3, 26, 14, 8, 12, 2, 20, 3, 11, 2, 13, 10, 12, 7, 27, 2, 3, 12, 5, 3, 7, 12, 8, 3, 20, 10, 28, 5, 11, 3, 23, 13, 10, 6, 2, 12, 3, 10, 6, 19, 3, 13, 10, 11, 25, 2, 11, 3, 12, 1, 10, 6, 3, 10, 6, 14, 3, 29, 6, 5, 24, 6, 3, 19, 24, 10, 11, 21, 3, 23, 13, 10, 6, 2, 12], [0, 1, 2, 3, 4, 5, 5, 6, 3, 7, 8, 3, 10, 3, 5, 1

In [89]:
# Find the length of each sentence
lengths = []
for sentence in int_sentences:
    lengths.append(len(sentence))
lengths = pd.DataFrame(lengths, columns=["counts"])

In [90]:
lengths.describe()

,counts
count,225857.000000
mean,152.020230
std,201.798021
min,1.000000
25%,84.000000
50%,123.000000
75%,177.000000
max,12459.000000


In [94]:
# Limit the data we will use to train our model
max_length = 200 # 92
min_length = 10

int_sentences = list(filter(lambda x: len(x) <= max_length and len(x) >= min_length, int_sentences))
int_noise_sentences = list(filter(lambda x: len(x) <= max_length and len(x) >= min_length, int_noise_sentences))

if len(int_sentences) != len(int_noise_sentences):
  raise Exception("can not map samples")

data = list(zip(int_noise_sentences, int_sentences))

print("We will use {} to train and test our model.".format(len(data)))

We will use 183664 to train and test our model.


In [95]:
print(data[2])

([34, 12, 3, 13, 10, 22, 29, 8, 3, 10, 6, 14, 3, 8, 7, 25, 6, 7, 21, 7, 22, 10, 6, 12, 3, 10, 12, 20, 5, 8, 23, 1, 2, 11, 2, 3, 1, 14, 19, 11, 5, 8, 23, 1, 2, 11, 2, 3, 5, 11, 3, 20, 10, 25, 6, 2, 12, 7, 22, 3, 21, 7, 2, 13, 19], [34, 12, 3, 13, 10, 22, 29, 8, 3, 10, 6, 14, 3, 8, 7, 25, 6, 7, 21, 7, 22, 10, 6, 12, 3, 10, 12, 20, 5, 8, 23, 1, 2, 11, 2, 3, 1, 14, 19, 11, 5, 8, 23, 1, 2, 11, 2, 3, 5, 11, 3, 20, 10, 25, 6, 2, 12, 7, 22, 3, 21, 7, 2, 13, 19])


In [96]:
# Split the data into training and testing sentences
training, testing = train_test_split(data, test_size = 0.2, random_state = 2)

print("Number of training sentences:", len(training))
print("Number of testing sentences:", len(testing))

Number of training sentences: 146931
Number of testing sentences: 36733


In [110]:
training[0]

([43,
  2,
  3,
  1,
  10,
  19,
  3,
  13,
  2,
  10,
  11,
  6,
  2,
  19,
  3,
  12,
  1,
  2,
  3,
  6,
  10,
  20,
  2,
  8,
  3,
  10,
  6,
  19,
  3,
  22,
  5,
  15,
  13,
  19,
  3,
  10,
  8,
  8,
  5,
  22,
  7,
  10,
  12,
  2,
  3,
  18,
  14,
  3,
  27,
  2,
  11,
  18,
  10,
  13,
  3,
  22,
  5,
  20,
  20,
  10,
  6,
  19,
  3,
  5,
  27,
  2,
  11,
  3,
  24,
  5,
  11,
  19,
  8],
 [43,
  2,
  3,
  1,
  10,
  19,
  3,
  13,
  2,
  10,
  11,
  6,
  2,
  19,
  3,
  12,
  1,
  2,
  3,
  6,
  10,
  20,
  2,
  8,
  3,
  10,
  6,
  19,
  3,
  22,
  5,
  15,
  13,
  19,
  3,
  10,
  8,
  8,
  5,
  22,
  7,
  10,
  12,
  2,
  3,
  18,
  14,
  3,
  27,
  2,
  11,
  18,
  10,
  13,
  3,
  22,
  5,
  20,
  20,
  10,
  6,
  19,
  3,
  5,
  27,
  2,
  11,
  3,
  24,
  5,
  11,
  19,
  8])

# Building the Model

## Build model

In [237]:
def model_inputs():
    '''Create palceholders for inputs to the model'''
    
    with tf.name_scope('inputs'):
        inputs = tf.placeholder(tf.int32, [None, None], name='inputs')
    with tf.name_scope('targets'):
        targets = tf.placeholder(tf.int32, [None, None], name='targets')
    keep_prob = tf.placeholder(tf.float32, name='keep_prob')
    inputs_length = tf.placeholder(tf.int32, (None,), name='inputs_length')
    targets_length = tf.placeholder(tf.int32, (None,), name='targets_length')
    max_target_length = tf.reduce_max(targets_length, name='max_target_len')

    return inputs, targets, keep_prob, inputs_length, targets_length, max_target_length

In [238]:
def process_encoding_input(targets, vocab_to_int, batch_size):
    '''Remove the last word id from each batch and concat the <GO> to the begining of each batch'''
    
    with tf.name_scope("process_encoding"):
        ending = tf.strided_slice(targets, [0, 0], [batch_size, -1], [1, 1])
        dec_input = tf.concat([tf.fill([batch_size, 1], vocab_to_int['<GO>']), ending], 1)

    return dec_input

In [239]:
def encoding_layer(rnn_size, sequence_length, num_layers, rnn_inputs, keep_prob, direction):
    '''Create the encoding layer'''
    
    if direction == 1:
        with tf.name_scope("RNN_Encoder_Cell_1D"):
            for layer in range(num_layers):
                with tf.variable_scope('encoder_{}'.format(layer)):
                    lstm = tf.contrib.rnn.LSTMCell(rnn_size)

                    drop = tf.contrib.rnn.DropoutWrapper(lstm, 
                                                         input_keep_prob = keep_prob)

                    enc_output, enc_state = tf.nn.dynamic_rnn(drop, 
                                                              rnn_inputs,
                                                              sequence_length,
                                                              dtype=tf.float32)

            return enc_output, enc_state
        
        
    if direction == 2:
        with tf.name_scope("RNN_Encoder_Cell_2D"):
            for layer in range(num_layers):
                with tf.variable_scope('encoder_{}'.format(layer)):
                    cell_fw = tf.contrib.rnn.LSTMCell(rnn_size)
                    cell_fw = tf.contrib.rnn.DropoutWrapper(cell_fw, 
                                                            input_keep_prob = keep_prob)

                    cell_bw = tf.contrib.rnn.LSTMCell(rnn_size)
                    cell_bw = tf.contrib.rnn.DropoutWrapper(cell_bw, 
                                                            input_keep_prob = keep_prob)

                    enc_output, enc_state = tf.nn.bidirectional_dynamic_rnn(cell_fw, 
                                                                            cell_bw, 
                                                                            rnn_inputs,
                                                                            sequence_length,
                                                                            dtype=tf.float32)
            # Join outputs since we are using a bidirectional RNN
            enc_output = tf.concat(enc_output,2)
            # Use only the forward state because the model can't use both states at once
            return enc_output, enc_state[0]

In [240]:
def training_decoding_layer(dec_embed_input, targets_length, dec_cell, initial_state, output_layer, 
                            vocab_size, max_target_length):
    '''Create the training logits'''
    
    with tf.name_scope("Training_Decoder"):
        training_helper = tf.contrib.seq2seq.TrainingHelper(inputs=dec_embed_input,
                                                            sequence_length=targets_length,
                                                            time_major=False)

        training_decoder = tf.contrib.seq2seq.BasicDecoder(dec_cell,
                                                           training_helper,
                                                           initial_state,
                                                           output_layer) 

        training_logits, _ = tf.contrib.seq2seq.dynamic_decode(training_decoder,
                                                               output_time_major=False,
                                                               impute_finished=True,
                                                               maximum_iterations=max_target_length)
        return training_logits

In [241]:
def inference_decoding_layer(embeddings, start_token, end_token, dec_cell, initial_state, output_layer,
                             max_target_length, batch_size):
    '''Create the inference logits'''
    
    with tf.name_scope("Inference_Decoder"):
        start_tokens = tf.tile(tf.constant([start_token], dtype=tf.int32), [batch_size], name='start_tokens')

        inference_helper = tf.contrib.seq2seq.GreedyEmbeddingHelper(embeddings,
                                                                    start_tokens,
                                                                    end_token)

        inference_decoder = tf.contrib.seq2seq.BasicDecoder(dec_cell,
                                                            inference_helper,
                                                            initial_state,
                                                            output_layer)

        inference_logits, _ = tf.contrib.seq2seq.dynamic_decode(inference_decoder,
                                                                output_time_major=False,
                                                                impute_finished=True,
                                                                maximum_iterations=max_target_length)

        return inference_logits

In [242]:
def decoding_layer(dec_embed_input, embeddings, enc_output, enc_state, vocab_size, inputs_length, targets_length, 
                   max_target_length, rnn_size, vocab_to_int, keep_prob, batch_size, num_layers, direction):
    '''Create the decoding cell and attention for the training and inference decoding layers'''
    
    with tf.name_scope("RNN_Decoder_Cell"):
        for layer in range(num_layers):
            with tf.variable_scope('decoder_{}'.format(layer)):
                lstm = tf.contrib.rnn.LSTMCell(rnn_size)
                dec_cell = tf.contrib.rnn.DropoutWrapper(lstm, 
                                                         input_keep_prob = keep_prob)
    
    output_layer = Dense(vocab_size,
                         kernel_initializer = tf.truncated_normal_initializer(mean = 0.0, stddev=0.1))
    
    attn_mech = tf.contrib.seq2seq.BahdanauAttention(rnn_size,
                                                  enc_output,
                                                  inputs_length,
                                                  normalize=False,
                                                  name='BahdanauAttention')
    
    with tf.name_scope("Attention_Wrapper"):
        dec_cell = tf.contrib.seq2seq.DynamicAttentionWrapper(dec_cell,
                                                              attn_mech,
                                                              rnn_size)
    
    initial_state = tf.contrib.seq2seq.DynamicAttentionWrapperState(enc_state,
                                                                    _zero_state_tensors(rnn_size, 
                                                                                        batch_size, 
                                                                                        tf.float32))

    with tf.variable_scope("decode"):
        training_logits = training_decoding_layer(dec_embed_input, 
                                                  targets_length, 
                                                  dec_cell, 
                                                  initial_state,
                                                  output_layer,
                                                  vocab_size, 
                                                  max_target_length)
    with tf.variable_scope("decode", reuse=True):
        inference_logits = inference_decoding_layer(embeddings,  
                                                    vocab_to_int['<GO>'], 
                                                    vocab_to_int['<EOS>'],
                                                    dec_cell, 
                                                    initial_state, 
                                                    output_layer,
                                                    max_target_length,
                                                    batch_size)

    return training_logits, inference_logits

In [243]:
def seq2seq_model(inputs, targets, keep_prob, inputs_length, targets_length, max_target_length, 
                  vocab_size, rnn_size, num_layers, vocab_to_int, batch_size, embedding_size, direction):
    '''Use the previous functions to create the training and inference logits'''
    
    enc_embeddings = tf.Variable(tf.random.uniform([vocab_size, embedding_size], -1, 1))
    enc_embed_input = tf.nn.embedding_lookup(enc_embeddings, inputs)
    enc_output, enc_state = encoding_layer(rnn_size, inputs_length, num_layers, 
                                           enc_embed_input, keep_prob, direction)
    
    dec_embeddings = tf.Variable(tf.random.uniform([vocab_size, embedding_size], -1, 1))
    dec_input = process_encoding_input(targets, vocab_to_int, batch_size)
    dec_embed_input = tf.nn.embedding_lookup(dec_embeddings, dec_input)
    
    training_logits, inference_logits  = decoding_layer(dec_embed_input, 
                                                        dec_embeddings,
                                                        enc_output,
                                                        enc_state, 
                                                        vocab_size, 
                                                        inputs_length, 
                                                        targets_length, 
                                                        max_target_length,
                                                        rnn_size, 
                                                        vocab_to_int, 
                                                        keep_prob, 
                                                        batch_size,
                                                        num_layers,
                                                        direction)
    
    return training_logits, inference_logits

In [244]:
def pad_sentence_batch(sentence_batch):
    """Pad sentences with <PAD> so that each sentence of a batch has the same length"""
    max_sentence = max([len(sentence) for sentence in sentence_batch])
    return [sentence + [vocab_to_int['<PAD>']] * (max_sentence - len(sentence)) for sentence in sentence_batch]

In [245]:
def get_batches(sentences, batch_size, threshold):
    """Batch sentences, noisy sentences, and the lengths of their sentences together.
       With each epoch, sentences will receive new mistakes"""
    
    for batch_i in range(0, len(sentences)//batch_size):
        start_i = batch_i * batch_size
        sentences_batch = sentences[start_i:start_i + batch_size]
        
        sentences_batch_noisy = []
        for sentence in sentences_batch:
            sentences_batch_noisy.append(sentence[0])
            
        sentences_batch_eos = []
        for sentence in sentences_batch:
            sentence[1].append(vocab_to_int['<EOS>'])
            sentences_batch_eos.append(sentence[1])
            
        pad_sentences_batch = np.array(pad_sentence_batch(sentences_batch_eos))
        pad_sentences_noisy_batch = np.array(pad_sentence_batch(sentences_batch_noisy))
        
        # Need the lengths for the _lengths parameters
        pad_sentences_lengths = []
        for sentence in pad_sentences_batch:
            pad_sentences_lengths.append(len(sentence))
        
        pad_sentences_noisy_lengths = []
        for sentence in pad_sentences_noisy_batch:
            pad_sentences_noisy_lengths.append(len(sentence))
        
        yield pad_sentences_noisy_batch, pad_sentences_batch, pad_sentences_noisy_lengths, pad_sentences_lengths

*Note: This set of values achieved the best results.*

In [246]:
# The default parameters
epochs = 100
batch_size = 128
num_layers = 2
rnn_size = 512
embedding_size = 128
learning_rate = 0.0005
direction = 2
threshold = 0.95
keep_probability = 0.75

In [247]:
def build_graph(keep_prob, rnn_size, num_layers, batch_size, learning_rate, embedding_size, direction):

    tf.reset_default_graph()    

    # Load the model inputs    
    inputs, targets, keep_prob, inputs_length, targets_length, max_target_length = model_inputs()

    # Create the training and inference logits
    training_logits, inference_logits = seq2seq_model(tf.reverse(inputs, [-1]),
                                                      targets, 
                                                      keep_prob,   
                                                      inputs_length,
                                                      targets_length,
                                                      max_target_length,
                                                      len(vocab_to_int)+1,
                                                      rnn_size, 
                                                      num_layers, 
                                                      vocab_to_int,
                                                      batch_size,
                                                      embedding_size,
                                                      direction)

    # Create tensors for the training logits and inference logits
    training_logits = tf.identity(training_logits.rnn_output, 'logits')

    with tf.name_scope('predictions'):
        predictions = tf.identity(inference_logits.sample_id, name='predictions')
        tf.summary.histogram('predictions', predictions)

    # Create the weights for sequence_loss
    masks = tf.sequence_mask(targets_length, max_target_length, dtype=tf.float32, name='masks')
    
    with tf.name_scope("cost"):
        # Loss function
        cost = tf.contrib.seq2seq.sequence_loss(training_logits, 
                                                targets, 
                                                masks)
        tf.summary.scalar('cost', cost)

    with tf.name_scope("optimze"):
        optimizer = tf.train.AdamOptimizer(learning_rate)

        # Gradient Clipping
        gradients = optimizer.compute_gradients(cost)
        capped_gradients = [(tf.clip_by_value(grad, -5., 5.), var) for grad, var in gradients if grad is not None]
        train_op = optimizer.apply_gradients(capped_gradients)

    # Merge all of the summaries
    merged = tf.summary.merge_all()    

    # Export the nodes 
    export_nodes = ['inputs', 'targets', 'keep_prob', 'cost', 'inputs_length', 'targets_length',
                    'predictions', 'merged', 'train_op','optimizer']
    Graph = namedtuple('Graph', export_nodes)
    local_dict = locals()
    graph = Graph(*[local_dict[each] for each in export_nodes])

    return graph

## Training the Model

In [248]:
def train(model, epochs, log_string):
    '''Train the RNN'''
    
    with tf.Session() as sess:
        sess.run(tf.global_variables_initializer())

        # Used to determine when to stop the training early
        testing_loss_summary = []

        # Keep track of which batch iteration is being trained
        iteration = 0
        
        display_step = 30 # The progress of the training will be displayed after every 30 batches
        stop_early = 0 
        stop = 3 # If the batch_loss_testing does not decrease in 3 consecutive checks, stop training
        per_epoch = 3 # Test the model 3 times per epoch
        testing_check = (len(training)//batch_size//per_epoch)-1

        print()
        print("Training Model: {}".format(log_string))

        train_writer = tf.summary.FileWriter('./logs/1/train/{}'.format(log_string), sess.graph)
        test_writer = tf.summary.FileWriter('./logs/1/test/{}'.format(log_string))

        for epoch_i in range(1, epochs+1): 
            batch_loss = 0
            batch_time = 0
            
            for batch_i, (input_batch, target_batch, input_length, target_length) in enumerate(
                    get_batches(training, batch_size, threshold)):
                start_time = time.time()

                summary, loss, _ = sess.run([model.merged,
                                             model.cost, 
                                             model.train_op], 
                                             {model.inputs: input_batch,
                                              model.targets: target_batch,
                                              model.inputs_length: input_length,
                                              model.targets_length: target_length,
                                              model.keep_prob: keep_probability})


                batch_loss += loss
                end_time = time.time()
                batch_time += end_time - start_time

                # Record the progress of training
                train_writer.add_summary(summary, iteration)

                iteration += 1

                if batch_i % display_step == 0 and batch_i > 0:
                    print('Epoch {:>3}/{} Batch {:>4}/{} - Loss: {:>6.3f}, Seconds: {:>4.2f}'
                          .format(epoch_i,
                                  epochs, 
                                  batch_i, 
                                  len(training) // batch_size, 
                                  batch_loss / display_step, 
                                  batch_time))
                    batch_loss = 0
                    batch_time = 0

                #### Testing ####
                if batch_i % testing_check == 0 and batch_i > 0:
                    batch_loss_testing = 0
                    batch_time_testing = 0
                    for batch_i, (input_batch, target_batch, input_length, target_length) in enumerate(
                            get_batches(testing_sorted, batch_size, threshold)):
                        start_time_testing = time.time()
                        summary, loss = sess.run([model.merged,
                                                  model.cost], 
                                                     {model.inputs: input_batch,
                                                      model.targets: target_batch,
                                                      model.inputs_length: input_length,
                                                      model.targets_length: target_length,
                                                      model.keep_prob: 1})

                        batch_loss_testing += loss
                        end_time_testing = time.time()
                        batch_time_testing += end_time_testing - start_time_testing

                        # Record the progress of testing
                        test_writer.add_summary(summary, iteration)

                    n_batches_testing = batch_i + 1
                    print('Testing Loss: {:>6.3f}, Seconds: {:>4.2f}'
                          .format(batch_loss_testing / n_batches_testing, 
                                  batch_time_testing))
                    
                    batch_time_testing = 0

                    # If the batch_loss_testing is at a new minimum, save the model
                    testing_loss_summary.append(batch_loss_testing)
                    if batch_loss_testing <= min(testing_loss_summary):
                        print('New Record!') 
                        stop_early = 0
                        checkpoint = "./{}.ckpt".format(log_string)
                        saver = tf.train.Saver()
                        saver.save(sess, checkpoint)

                    else:
                        print("No Improvement.")
                        stop_early += 1
                        if stop_early == stop:
                            break

            if stop_early == stop:
                print("Stopping Training.")
                break

## Train

In [ ]:
# Train the model with the desired tuning parameters
for keep_probability in [0.75]:
    for num_layers in [2]:
        for threshold in [0.95]:
            log_string = 'kp={},nl={},th={}'.format(keep_probability,
                                                    num_layers,
                                                    threshold) 
            model = build_graph(keep_probability, rnn_size, num_layers, batch_size, 
                                learning_rate, embedding_size, direction)
            train(model, epochs, log_string)

## Fixing Custom Sentences

In [ ]:
def text_to_ints(text):
    '''Prepare the text for the model'''
    
    text = clean_text(text)
    return [vocab_to_int[word] for word in text]

In [ ]:
# Create your own sentence or use one from the dataset
text = "Spellin is difficult, whch is wyh you need to study everyday."
text = text_to_ints(text)

#random = np.random.randint(0,len(testing_sorted))
#text = testing_sorted[random]
#text = noise_maker(text, 0.95)

checkpoint = "./kp=0.75,nl=2,th=0.95.ckpt"

model = build_graph(keep_probability, rnn_size, num_layers, batch_size, learning_rate, embedding_size, direction) 

with tf.Session() as sess:
    # Load saved model
    saver = tf.train.Saver()
    saver.restore(sess, checkpoint)
    
    #Multiply by batch_size to match the model's input parameters
    answer_logits = sess.run(model.predictions, {model.inputs: [text]*batch_size, 
                                                 model.inputs_length: [len(text)]*batch_size,
                                                 model.targets_length: [len(text)+1], 
                                                 model.keep_prob: [1.0]})[0]

# Remove the padding from the generated sentence
pad = vocab_to_int["<PAD>"] 

print('\nText')
print('  Word Ids:    {}'.format([i for i in text]))
print('  Input Words: {}'.format("".join([int_to_vocab[i] for i in text])))

print('\nSummary')
print('  Word Ids:       {}'.format([i for i in answer_logits if i != pad]))
print('  Response Words: {}'.format("".join([int_to_vocab[i] for i in answer_logits if i != pad])))

INFO:tensorflow:Restoring parameters from ./lr=0.75,nl=2,th=0.95.ckpt

Text
  Word Ids:    [53, 9, 23, 20, 20, 7, 5, 19, 7, 6, 19, 22, 7, 2, 2, 7, 8, 16, 20, 1, 39, 19, 28, 24, 8, 24, 19, 7, 6, 19, 28, 40, 24, 19, 40, 13, 16, 19, 5, 23, 23, 22, 19, 1, 13, 19, 6, 1, 16, 22, 40, 19, 23, 27, 23, 0, 40, 22, 4, 40, 42, 19]
  Input Words: Spellin is difficult, whch is wyh you need to study everyday. 

Summary
  Word Ids:       [53, 9, 23, 20, 20, 7, 5, 10, 19, 7, 6, 19, 22, 7, 2, 2, 7, 8, 16, 20, 1, 39, 19, 28, 24, 7, 8, 24, 19, 7, 6, 19, 28, 24, 40, 19, 40, 13, 16, 19, 5, 23, 23, 22, 19, 1, 13, 19, 6, 1, 16, 22, 40, 19, 23, 27, 23, 0, 40, 22, 4, 40, 42]
  Response Words: Spelling is difficult, which is why you need to study everyday.
